# Test functions


In [1]:
import psr.factory
import psr.outputs
import traceback
from pathlib import Path
CASE_PATH = r"C:\PSR\SDDP18.0\examples\operation\1_stage\Case01"
STUDY = psr.factory.load_study(CASE_PATH)

In [6]:
def list_enabled_outputs():
    """
    List all currently enabled outputs in the study case.

    This tool returns a dictionary containing only the outputs that are currently
    enabled (active) in the study, mapping each output filename to its description.

    This function should be used when:
        - The user asks which outputs are currently active or available.
        - The agent needs to inspect which outputs can be accessed or converted.
        - As a validation step before calling `convert_output`.

    Usage:
        - Call this function to retrieve the list of enabled outputs.
        - If file paths are required, use `list_enabled_outputs_paths` instead.

    Returns:
        dict[str, str]: A dictionary mapping output filenames to their descriptions.
    """
    try: 
        case_path = CASE_PATH
        outputs = {}
        outputs_list = psr.outputs.get_available_outputs(case_path)
        for output in outputs_list:
            outputs[output.filename]=output.description
        return outputs
    except Exception as e:
        tb = traceback.format_exc()
        return f"TOOL_ERROR: list_enabled_outputs  failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\n."


print(list_enabled_outputs())

{'demand': 'Load per system - inelastic', 'defcos': 'Deficit cost', 'defcit': 'Deficit', 'gerter': 'Thermal generation', 'cmgdem': 'Load marginal cost', 'duraci': 'Load level length', 'demandel': 'Load per system - supplied', 'demxclel': 'Load per class - supplied', 'objcop': 'Costs by category', 'duracipu': 'Block length in pu', 'sddp_dashboard_cost_avg': 'Average operating costs per stage', 'sddp_dashboard_cost_disp': 'Dispersion of operating costs per stage', 'sddp_dashboard_cost_tot': 'Breakdown of total operating costs', 'sddp_dashboard_rev_disp': 'Dispersion of revenues per stage', 'sddprisk': 'Deficit risk per system'}


In [ ]:
def list_enabled_outputs_paths():
    """
    List full file paths of all currently enabled outputs in the study case.

    This tool returns a dictionary mapping each enabled output's absolute file path
    to its description.

    This function should be used when:
        - The agent needs the physical file path for reading, converting or exporting outputs.
        - Before calling `convert_output`, to validate valid input paths.

    Usage:
        - Call this function to retrieve all enabled output paths.
        - Select the desired file path and pass it to `convert_output`.

    Returns:
        dict[str, str]: A dictionary mapping full file paths to output descriptions.
    """
    try:
        case_path = CASE_PATH
        outputs_list = psr.outputs.get_available_outputs(case_path)
        paths = {}
        for output in outputs_list:
            filename = output.filename 
            ext = output.file_type
            path = str(Path(case_path) / f"{filename}.{ext}")
            paths[path] = output.description
        return paths
    except Exception as e:
        tb = traceback.format_exc()
        return f"TOOL_ERROR: list_enabled_outputs_paths: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\n."

print(list_enabled_outputs_paths())

{'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\demand.csv': 'Load per system - inelastic', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\defcos.csv': 'Deficit cost', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\defcit.csv': 'Deficit', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\gerter.csv': 'Thermal generation', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\cmgdem.csv': 'Load marginal cost', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\duraci.csv': 'Load level length', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\demandel.csv': 'Load per system - supplied', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\demxclel.csv': 'Load per class - supplied', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\objcop.csv': 'Costs by category', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\duracipu.csv': 'Block length in pu', 'C:\\PSR\\SDDP18.0\\examples\\operation\\1_stage\\Case01\\

In [ ]:
def convert_output(file_path, format):
    """Convert an output file to a different format.
    
    Converts the specified output file to the requested file format.
    
    Args:
        file_path (str): The path to the output file to convert.
        format (str): The target file format (e.g., 'csv', 'json', 'xlsx').
    
    Returns:
        bool: True if conversion was successful.
    """
    try: 
        p = Path(file_path)
        new_path = str(p.with_suffix(f".{format}"))
        psr.factory.convert_output(file_path,new_path)
        return "Converted with succes"
    except Exception as e:
        tb = traceback.format_exc()
        return f"TOOL_ERROR: convert_output failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\nSuggested action: verify with the filepath exist. Call list_available_outputs_paths first."


In [ ]:
case_path = CASE_PATH

def get_output_num():
    """
    Retrieve all available outputs and their descriptions.

    This tool returns a dictionary mapping each output ID (Num) to its textual
    description. It must be used as an intermediate step to interpret the user's
    query and identify which outputs should be enabled or disabled.

    Usage:
        - Call this function to obtain all (id → description) pairs.
        - Compare the user query semantically against the descriptions.
        - Select the output IDs whose descriptions best match the user request.
        - Pass the selected IDs to `change_output_availability`.

    Typical examples:
        - User: "Enable marginal cost outputs"
          → Match query with descriptions, find corresponding Num values,
            then call change_output_availability.

        - User: "Disable hydro generation reports"
          → Identify matching descriptions and extract their Num.

    Returns:
        dict[int, str]: A dictionary mapping output ID (Num) to output description.
    """
    try:
        case_path = CASE_PATH
        df = psr.outputs.load_index_dat(case_path)
        d = df.set_index('Num')['Description'].to_dict()
        return str(d)
    except Exception as e:
        tb = traceback.format_exc()
        #logger.error(f"TOOL_ERROR: get_output_num failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\nSuggested action: verify if the study case path exists, the outputs index file is accessible, and call list_available_outputs_paths first.")
        return f"TOOL_ERROR: get_output_num failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\nSuggested action: verify if the study case path exists, the outputs index file is accessible, and call list_available_outputs_paths first."

    
print(get_output_num())



{1: 'Alert storage', 2: 'Available hydro capacity', 3: 'Available thermal capacity', 4: 'Historic inflows average', 5: 'Inflows', 6: 'Load per system - inelastic', 7: 'Load per bus - inelastic', 8: 'Maximum storage', 9: 'Maximum stored energy', 10: 'Maximum total outflow', 11: 'Maximum turbined outflow', 12: 'Minimum security storage', 13: 'Minimum total outflow', 14: 'Minimum turbined outflow', 15: 'Deficit cost', 16: 'Flag for bus deficit', 17: 'Flag for qmin violation', 18: 'Flag for system deficit', 19: 'Penalty cost of spillage', 20: 'Thermal operating cost', 21: 'Viol. cost of alert storage', 22: 'Viol. cost of max.tot.outflow', 23: 'Viol. cost of min.op.storage', 24: 'Viol. cost of min.tot.outflow', 25: 'AC Circuit flow', 26: 'Deficit', 27: 'Deficit per bus', 28: 'Export by area', 29: 'Final head', 30: 'Final storage', 31: 'Fuel consumption', 32: 'Fuel consumption rate', 33: 'Hydro generation', 34: 'Interconnection flow', 35: 'Spilled outflow', 36: 'Stored energy', 37: 'Thermal 

In [9]:
case_path = CASE_PATH
def change_output_availability(outputs:dict[int,bool]):
    """
    Enable or disable study outputs based on their IDs.

    This tool updates the availability status of one or more outputs in the study.
    The input must be a dictionary mapping output IDs (Num) to boolean values,
    where:

        - True  → enable the output
        - False → disable the output

    This function must always be called after identifying the correct output IDs
    using `get_output_num`, by matching the user query against the output
    descriptions.

    Usage:
        - First, call `get_output_num()` to retrieve all (id → description) pairs.
        - Determine which outputs match the user's request.
        - Build a dictionary {Num: action}.
        - Call this function to apply the changes.

    Example:
        User: "Disable thermal generation and deficit outputs"

        Step 1: Call get_output_num()
        Step 2: Identify matching IDs, e.g. {3, 7}
        Step 3: Call:
            change_output_availability({
                3: False,
                7: False
            })

    Args:
        outputs (dict[int, bool]): Dictionary mapping output ID (Num) to action:
                                   True to enable, False to disable.

    Returns:
        pandas.DataFrame: Updated outputs table with modified availability flags.
    """
    try: 
        df_out = psr.outputs.load(CASE_PATH)
        result = {}
        for num, action in outputs.items():
            num = float(num)
            filter = df_out['Num'] == num
            df_out.loc[filter, 'Active'] = action
            result.update(df_out.loc[filter, 'Active'].to_dict())
        psr.outputs.save(df_out,case_path=CASE_PATH)
        return result
    except Exception as e:
        tb = traceback.format_exc()
        #logger.error(f"TOOL_ERROR: change_output_availability failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\nSuggested action: confirm that the provided output IDs are valid by calling get_output_num first.")
        return f"TOOL_ERROR: change_output_availability failed: {type(e).__name__}: {str(e)}\nTraceback:\n{tb}\nSuggested action: confirm that the provided output IDs are valid by calling get_output_num first."

print(psr.outputs.load(case_path))
print(change_output_availability({1:False, 2: False}))
print(psr.outputs.load(case_path))

           Num Active
volale     1.0  False
pothid     2.0  False
potter     3.0  False
histav     4.0  False
inflow     5.0  False
...        ...    ...
rsbnexc  436.0  False
gncivio  437.0  False
gncivic  438.0  False
occterm  450.0  False
enveper  457.0  False

[363 rows x 2 columns]
{'volale': False, 'pothid': False}
           Num Active
volale     1.0  False
pothid     2.0  False
potter     3.0  False
histav     4.0  False
inflow     5.0  False
...        ...    ...
rsbnexc  436.0  False
gncivio  437.0  False
gncivic  438.0  False
occterm  450.0  False
enveper  457.0  False

[363 rows x 2 columns]
